In [12]:
import pandas as pd

# Merging SDES database with Profeel to get households information
The method is to match the sdes archetypes to a Profeel one according to the values for the three index levels: Housing types, Energy performance, heating energy.
And then we calculate among each group the weight of the sdes archetypes in terms of their sdes's size estimate. Finally, this weights allow us to get the size of each lines in terms of Profeel stock.

The merging rule is to conserve, from the database of origin (SDES), the proportions of each household characteristics among a group of buildings which have same basic building characteristics.

In [13]:
profeel = pd.read_csv('output/building_stock_profeel_std.csv', index_col='Housing type')
sdes = pd.read_csv('building_stock_sdes2018_aggregated.csv', index_col=['Housing type', 'Energy performance', 'Heating energy'])

profeel = profeel.rename(columns={'calculated_epc':'Energy performance'})
profeel['Heating energy'] = profeel['Heating system'].str.split('-').str[0]
profeel = profeel.set_index(['Heating energy', 'Energy performance'], append=True)

In [14]:
merged = profeel.merge(sdes, how='inner', on=['Housing type', 'Energy performance', 'Heating energy'])
# Adding a row specific index level
r_position = pd.Series(range(0,merged.shape[0]), name='r_position')
merged = merged.set_index(r_position, append=True)

# Weight of each lines in terms of sdes's sizes (y) among a profeel archetype uniquely define by its class label
merged = merged.set_index('Class', append=True)
merged['Stock buildings_y'] = merged['Stock buildings_y'].T / merged.groupby(level='Class')['Stock buildings_y'].sum()

# Multiplying the weights to the profeel's stocks (x)
merged['Stock buildings'] = (merged['Stock buildings_y'].T * merged['Stock buildings_x']).T
cleaned = merged.reset_index(('Energy performance', 'r_position', 'Class', 'Heating energy'))
cleaned = cleaned.drop(labels=['Stock buildings_x', 'Stock buildings_y', 'Energy performance', 'r_position', 'Class', 'Heating energy'], axis=1)
cleaned

,Heating system,Wall,Floor,Roof,Windows,Occupancy status,Income owner,Income tenant,Stock buildings
Housing type,,,,,,,,,
Single-family,Wood fuel-Standard boiler,2.50,2.00,2.5,2.4,Social-housing,D10,D1,966.576810
Single-family,Wood fuel-Standard boiler,2.50,2.00,2.5,2.4,Owner-occupied,D1,D1,43705.999858
Single-family,Wood fuel-Standard boiler,2.50,2.00,2.5,2.4,Social-housing,D10,D2,1012.146645
Single-family,Wood fuel-Standard boiler,2.50,2.00,2.5,2.4,Owner-occupied,D2,D2,55337.810299
Single-family,Wood fuel-Standard boiler,2.50,2.00,2.5,2.4,Social-housing,D10,D3,900.817446
...,...,...,...,...,...,...,...,...,...
Multi-family,Electricity-Performance boiler,0.34,0.25,0.2,1.6,Privately rented,D5,D10,2805.583667
Multi-family,Electricity-Performance boiler,0.34,0.25,0.2,1.6,Privately rented,D6,D10,3803.898886
Multi-family,Electricity-Performance boiler,0.34,0.25,0.2,1.6,Privately rented,D7,D10,5325.218225


In [15]:
# Checks
print("profeel building stock: {}".format(profeel['Stock buildings'].sum()))
print("sdes building stock: {}".format(sdes['Stock buildings'].sum()))
print("The number of new lines for each profeel archetypes: {}".format(merged.shape[0]/profeel.shape[0]))
print("merged building stock: {}".format(merged['Stock buildings_x'].sum()/(merged.shape[0]/profeel.shape[0])))
print("output building stock: {}".format(cleaned['Stock buildings'].sum()))
print("building stock loss: {}".format(profeel['Stock buildings'].sum() - cleaned['Stock buildings'].sum()))

profeel building stock: 35172600
sdes building stock: 26698690.691502385
The number of new lines for each profeel archetypes: 120.0
merged building stock: 35172600.0
output building stock: 35172600.0
building stock loss: 0.0


In [17]:
cleaned.to_csv('output/building_stock_profeel_final.csv')